# Dask-Gateway Connection Example

In this notebook, we will connect to the gateway, create a cluster, spin up some workers, and run a trivial computation.

In [ ]:
from dask_gateway import Gateway, BasicAuth
import getpass
import dask
from distributed import Client

## Authenticate to the Gateway

Let's create a basic auth object to connect to the cluster. There is no real authentication that needs to be done, so any password will work.

In [ ]:
password = getpass.getpass()

In [ ]:
auth = BasicAuth("pangeo", password)

The address you need in the quotes can be found with
```
kubectl get svc -n dask-gateway traefik-dask-gateway
```
The entry for `EXTERNAL-IP` is what you want after `http://`.

In [ ]:
gateway = Gateway("http://35.236.84.22", auth=auth)
gateway.list_clusters()

In [ ]:
# No cluster options are specified here, but this would show them if they existed.
gateway.cluster_options()

## Spin up Cluster

Once the cluster is running, we can ask it to provision workers.

In [ ]:
cluster = gateway.new_cluster()
client = Client(cluster, set_as_default=False)
client

In [ ]:
cluster.adapt(minimum=2,maximum=5)

The cell below should show that the cluster now has workers. To inspect the cluster further, you can click the Dask symbol on the left sidebar (hovering your mouse over it should make the word "Dask" pop up). From there, copy the dashboard link from the Client into the search bar, click the magnifying glass search button, and then click on the orange window buttons beneath to see more.

In [ ]:
# Check that the cluster has scaled
client

## Trivial Computation

In [ ]:
import dask.array as da

In [ ]:
a = da.random.normal(size=(1000,1000), chunks=(500,500))

In [ ]:
a.mean().compute()

## Shutdown the Cluster

In [ ]:
client.close()
cluster.close()